In [ ]:

synthesisOpToPosDic = {
    0: "refactor",
    1: "refactor -z",
    2: "rewrite",
    3: "rewrite -z",
    4: "resub",
    5: "resub -z",
    6: "balance"
}

AIG = 'alu4.aig'
libFile = './lib/7nm/7nm.lib'
logFile = 'alu4.log'

for step in range(10):
    childs = [] 
    for child in range(7):
        childFile = 'alu4_' + str(child) + '.aig'
        abcRunCmd = './yosys-abc -c "read ' + AIG + ';' + synthesisOpToPosDic[child] + '; read_lib ' + libFile + '; write ' + childFile + '; print_stats" > ' + logFile
        os.system(abcRunCmd)
        childs.append(childFile)
    childScores = Evaluation(childs)
    action = argmax(childScores)
    AIG = childs[action]

abcRunCmd = './yosys-abc -c "read ' + AIG + '; read_lib ' + libFile + '; map; topo; stime" > ' + logFile
os.system(abcRunCmd)

with open(logFile) as f:
    areaInformation = re.findall('[a-zA-Z0-9\.]+', f.readlines()[-1])
    adpVal = float(areaInformation[-9]) + float(areaInformation[-4])

return (baseline - adpVal) / baseline
